# 5강) BERT를 활용한 Dense Passage Retrieval 실습
*   KorQuAD train 데이터셋을 활용해서 Dense Encoder을 학습시킬 수 있다.

*   Dense Encoder를 통해 Dense Embedding을 만들 수 있다.

*   Dense Embedding을 활용하여 passage retrieval를 진행할 수 있다.


### Requirements

## 데이터셋 로딩


KorQuAD train 데이터셋을 학습 데이터로 활용

In [1]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [2]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [3]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [5]:
from tqdm.notebook import tqdm
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2023)
torch.cuda.manual_seed(2023)
np.random.seed(2023)
random.seed(2023)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [6]:
# Use subset (128 example) of original training dataset
sample_idx = np.random.choice(range(len(dataset['train'])), 3000)
training_dataset = dataset['train'][sample_idx]

In [7]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')


In [8]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [9]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()

  def forward(self, input_ids,
              attention_mask=None, token_type_ids=None):

      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)

      pooled_output = outputs[1]

      return pooled_output


In [10]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Train function 정의 후, 두개의 encoder fine-tuning 하기 (In-batch negative 활용)


In [11]:
def train(args, dataset, p_model, q_model):

  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)


  ### 추가 부분 ###
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  ### 추가 부분 ###

  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0

  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()

  # train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for i in range(int(args.num_train_epochs)):
    print(f"Epoch {i+1}/{args.num_train_epochs}")
    # epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(tqdm(train_dataloader)):
      q_encoder.train()
      p_encoder.train()

      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)

      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2]
                  }

      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}

      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)


      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

      # target: position of positive samples = diagonal element
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')

      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1

      torch.cuda.empty_cache()



  return p_model, q_model

In [12]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01
)

In [13]:
p_encoder, q_encoder = train(args, train_dataset, p_encoder, q_encoder)

Epoch 1/5


/data/ephemeral/home/miniforge3/envs/mrc/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/375 [00:00<?, ?it/s]

Epoch 2/5


  0%|          | 0/375 [00:00<?, ?it/s]

...

  0%|          | 0/375 [00:00<?, ?it/s]

Epoch 4/5


  0%|          | 0/375 [00:00<?, ?it/s]

Epoch 5/5


  0%|          | 0/375 [00:00<?, ?it/s]

In [ ]:
# 모델 저장하기
# model save
import os

dpr_folder_path = "../models/dpr/"

# 폴더가 없으면 생성
if not os.path.exists(dpr_folder_path):
    os.makedirs(dpr_folder_path)

p_encoder_model_path = os.path.join(dpr_folder_path, "p_encoder")
p_encoder.save_pretrained(p_encoder_model_path)

q_encoder_model_path = os.path.join(dpr_folder_path, "q_encoder")
q_encoder.save_pretrained(q_encoder_model_path)

## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [34]:
valid_corpus = list(set([example['context'] for example in dataset['validation']]))
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(f"Query: {query}")
print(f"Grount Truth: {ground_truth}")

Query: 폰테인을 무찌른 잭이 리틀 시스터에게 돌아서서 그들을 모두 죽이고 챙긴것으로 암시되는 것은 무엇인가?
Grount Truth: 두 번째로, 만약 플레이어가 모든 리틀 시스터로부터 ADAM을 채취했다면 (그러므로 죽였다면), 잭이 폰테인을 무찌른 후 리틀 시스터에게 돌아서며, 그들을 모두 죽이고 ADAM을 챙겼을 것으로 암시된다. 테넌바움은 경멸과 분노에 쌓인 목소리로 잭과 그의 행동을 비난한다. 그 뒤에, 핵 미사일을 장책한 탄도 미사일 장착 잠수함 주위에 구형 잠수기 여러개가 표면으로 올라오고, 스플라이서가 나오면서 사람들을 모두 죽이고 잠수함을 장악한다. 그 후, 핵 미사일이 등장하며 이것으로 잭이 세계대전을 일으킴을 알 수 있다. 세 번째로 플레이어가 몇 명은 살렸으나 조금은 죽였다면, 영상은 두 번째 엔딩과 같지만 목소리의 톤은 두 번째의 분노와는 달리 슬픈 어조를 띈다.


앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [35]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [36]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in tqdm(valid_corpus):
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

  0%|          | 0/960 [00:00<?, ?it/s]

torch.Size([960, 768]) torch.Size([1, 768])


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [37]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

torch.Size([1, 960])
tensor([[19.6556, 22.3597, 14.1968, 12.5722, 22.6637, 18.6194, 25.8153, 13.0454,
         19.0301, 16.4470, 16.6542, 17.1530, 24.2421, 15.6271, 16.2194, 11.7617,
         22.5374, 18.1069, 13.1937, 15.1691, 19.3456, 16.4855, 22.7450, 16.0518,
         22.4289, 16.3197, 18.0905, 26.7573, 14.8262, 17.3686, 18.7622, 19.9355,
         17.7241, 24.6848, 18.8314, 21.5743, 21.4829, 21.3272, 12.2493, 17.3133,
         23.4274, 15.1159, 24.2736, 16.1218, 20.9691, 20.4794, 22.4146, 18.7367,
         27.1128, 16.8777, 14.4584, 21.2524, 17.4445, 21.4694, 19.4052, 19.2344,
         14.1084, 14.0239, 21.5008, 13.6387, 20.5474, 16.0396, 17.5174, 22.8792,
         21.2981, 17.3243, 20.1221, 17.7216, 17.3160, 20.6238, 15.8987, 17.4906,
         15.6523, 17.6685, 13.9443, 14.9983, 18.5167, 23.4118, 19.9696, 15.8082,
         18.6093, 17.9288, 20.2531, 15.9631, 14.7658, 19.5599, 15.0578, 17.8662,
         18.1896, 19.2535, 18.0201, 19.6173, 17.2316, 21.3100, 18.2667, 13.7866,
       

Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [38]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 폰테인을 무찌른 잭이 리틀 시스터에게 돌아서서 그들을 모두 죽이고 챙긴것으로 암시되는 것은 무엇인가? 

[Ground truth passage]
두 번째로, 만약 플레이어가 모든 리틀 시스터로부터 ADAM을 채취했다면 (그러므로 죽였다면), 잭이 폰테인을 무찌른 후 리틀 시스터에게 돌아서며, 그들을 모두 죽이고 ADAM을 챙겼을 것으로 암시된다. 테넌바움은 경멸과 분노에 쌓인 목소리로 잭과 그의 행동을 비난한다. 그 뒤에, 핵 미사일을 장책한 탄도 미사일 장착 잠수함 주위에 구형 잠수기 여러개가 표면으로 올라오고, 스플라이서가 나오면서 사람들을 모두 죽이고 잠수함을 장악한다. 그 후, 핵 미사일이 등장하며 이것으로 잭이 세계대전을 일으킴을 알 수 있다. 세 번째로 플레이어가 몇 명은 살렸으나 조금은 죽였다면, 영상은 두 번째 엔딩과 같지만 목소리의 톤은 두 번째의 분노와는 달리 슬픈 어조를 띈다. 

Top-1 passage with score 30.8887
테넌바움 박사가 내레이션을 맡은 엔딩은 플레이어가 리틀 시스터를 어떻게 다뤘느냐에 따라 3가지로 나뉜다. 해피 엔딩과 두 가지의 배드 엔딩으로 나뉘는데, 첫 번째로 플레이어가 리틀 시스터를 전부 구했다면 (그러므로 그들이 잭의 도움으로 전부 살아남았고 게다가 리틀 시스터의 희생을 최대로 줄였다면) 잭이 폰테인을 무찌른 후 리틀 시스터에게 돌아서서 다정하게 대한 후 스플라이서를 전부 죽인 뒤 마지막까지 살아남은 5명의 리틀 시스터와 함께 랩쳐를 떠난 것으로 암시된다. 엔딩은 5명의 리틀 시스터가 잭의 양녀가 되어 그와 육지로 돌아가 그의 보살핌 아래 살게 되고, 대학을 졸업하고, 결혼하며, 아이를 갖게 된다는 내용을 보게 된다. 테넌바움은 행복감을 느끼는 목소리로 잭의 정의로운 행동을 칭찬하며, 성인이 된 5명의 리틀 시스터가 잭의 임종을 지켜보는 것으로 끝을 맺는다.
Top-2 passage with score 30.6593
게임은 1960년 랩쳐가 붕괴된 직후 잭(게임의 주

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License



In [44]:
def search(dataset, q_encoder, p_embs, k=5, verbose=False):
    with torch.no_grad():
        sample_idx = random.choice(range(len(dataset['validation'])))
        query = dataset['validation'][sample_idx]['question']
        ground_truth = dataset['validation'][sample_idx]['context']

        if not ground_truth in valid_corpus:
            valid_corpus.append(ground_truth)

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')

        dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
        rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

        if verbose:
            print("[Search query]\n", query, "\n")
            print("[Ground truth passage]")
            print(ground_truth, "\n")

            for i in range(k):
                print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
                print(valid_corpus[rank[i]])

        return [valid_corpus[rank[i]] for i in range(k)]

search(dataset, q_encoder, p_embs, k=5, verbose=True)

[Search query]
 승리가 해외 연예인으로는 최초로 진행을 맡은 일본 지상파 방송은? 

[Ground truth passage]
승리는 2013년 초, 그룹의 멤버 대성과 일본 활동에 주력하고자 도쿄 숙소에서 거주했다. 2013년 7월 28일, YG 엔터테인먼트는 승리의 두 번째 EP 음반 《Let's Talk About Love》을 8월 19일에 발매한다고 발표했다. 그는 9월 말까지 한국에서 자신의 앨범을 홍보했으며, 승리는 2013년 10월 9일에 그의 첫 번째 일본어 음반 《Let's Talk About Love》을 발표했다. 그의 일본어 음반에는 한국에서 이전에 발매 된 《Let's Talk About Love》의 수록 곡 외에도 그의 첫 EP 음반 《V.V.I.P》의 수록 곡들을 일본어로 새롭게 녹음했다. 또한 이 음반의 새로운 곡 "空に描く思い"(하늘에 그리는 생각)'은 승리가 주연을 맡은 모바일 앱(UULA)에서 독점 방송 된 모바일 드라마 《유비코이~ 그대에게 보내는 메시지》의 주제가로 기용되었다. 이 드라마에서 그는 주연을 맡아 여배우 타키모토 미오리와 혼고 카나타와 연기했으며, 이것은 승리에게 있어 최초의 일본어 연기였다. 2013년 9월 23일, 승리는 일본에서 후지 TV 《인기녀 100》이라는 파일럿 프로그램의 진행을 야마자키 히로시와 함께 맡았다. 이것은 해외 연예인으로 최초로 일본 지상파 방송의 진행을 맡은 것이다. 

Top-1 passage with score 19.5167
승리는 2012년 7월 쯤부터 일본 버라이어티 방송에 출연하기 시작하며 그의 솔로로서 첫 활동을 시작했다. 요미우리 TV의 《요시모토 죠네쯔 코메디 TV노 우라가와데 오오사와기! 몬스터AD 훈토키》와 후지 TV의 《사키가게 온카쿠 반즈케》의 스페셜 MC를 맡았다. 이 방송에서 그는 오구리 슌, EXILE의 아키라, 타키모토 미오리, 타케이 에미, Perfume 등 일본의 유명인들을 인터뷰했다. 그해 8월 2일에는 케이블 채널 Space Show TV

['승리는 2012년 7월 쯤부터 일본 버라이어티 방송에 출연하기 시작하며 그의 솔로로서 첫 활동을 시작했다. 요미우리 TV의 《요시모토 죠네쯔 코메디 TV노 우라가와데 오오사와기! 몬스터AD 훈토키》와 후지 TV의 《사키가게 온카쿠 반즈케》의 스페셜 MC를 맡았다. 이 방송에서 그는 오구리 슌, EXILE의 아키라, 타키모토 미오리, 타케이 에미, Perfume 등 일본의 유명인들을 인터뷰했다. 그해 8월 2일에는 케이블 채널 Space Show TV에서 《승리의 완전 승리 선언》이라는 자신의 이름을 건 프로그램을 진행했다. 2012년 8월 27일, 그는 도쿄에서 자신의 첫 번째 솔로 팬 미팅을 가졌다. 빅뱅의 멤버 대성이 무대에 올라 "Fantastic Baby"를 함께 부르며 서포트했다. 2012년 9월 9일에는 오사카에서 두 번째 솔로 팬미팅을 가졌는데, 팬미팅 사회는 승리의 완전 승리 선언 등 일본 방송활동 중에 연을 맺은 일본 개그맨들이 맡았다. 2012년 9월 1일에는 NTV 60주년 스페셜 드라마 《김전일 소년 사건부 - 홍콩구룡재보살인사건》에 승리가 합류하게 되었다는 기사가 언론에 발표되었다. 승리는 헤이! 세이! 점프의 야마다 료스케, 아리오카 다이키와 비비안 수 등과 출연했으며, 2013년 1월 12일에 방송 되었다. 이 작품은 2013 도쿄 드라마 어워드에서 최우수 드라마 SP(우수 작품상)을 수상했다.',
 '승리는 2013년 초, 그룹의 멤버 대성과 일본 활동에 주력하고자 도쿄 숙소에서 거주했다. 2013년 7월 28일, YG 엔터테인먼트는 승리의 두 번째 EP 음반 《Let\'s Talk About Love》을 8월 19일에 발매한다고 발표했다. 그는 9월 말까지 한국에서 자신의 앨범을 홍보했으며, 승리는 2013년 10월 9일에 그의 첫 번째 일본어 음반 《Let\'s Talk About Love》을 발표했다. 그의 일본어 음반에는 한국에서 이전에 발매 된 《Let\'s Talk About Love》의 수록 곡 외에도 그의 첫 EP

In [48]:
def evaluate(dataset, q_encoder, p_embs, k=5):
    with torch.no_grad():
        correct = 0
        for i in tqdm(range(len(dataset['validation']))):
            ground_truth = dataset['validation'][i]['context']
            query = dataset['validation'][i]['question']

            if not ground_truth in valid_corpus:
                valid_corpus.append(ground_truth)

            q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
            q_emb = q_encoder(**q_seqs_val).to('cpu')

            dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
            rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

            if ground_truth in [valid_corpus[rank[i]] for i in range(k)]:
                correct += 1

    return correct / len(dataset['validation'])

evaluate(dataset, q_encoder, p_embs, k=15)

  0%|          | 0/5774 [00:00<?, ?it/s]

0.835